In [10]:
import serial.tools.list_ports as Ports
from typing import List
import serial
import re
import time
    

## Identifying connections and ports

In [11]:

ports = Ports.comports()
for port in ports:
    print(f"Port: {port.device}, Description: {port.description}")


Port: /dev/cu.BLTH, Description: n/a
Port: /dev/cu.Bluetooth-Incoming-Port, Description: n/a
Port: /dev/cu.usbmodemN32G45x1, Description: N32G45x Port  


## SIYI Datalink SDK

| **Field**   | **Index** | **Bytes** | **Description**                                                                                                                        |
|-------------|-----------|-----------|----------------------------------------------------------------------------------------------------------------------------------------|
| **STX**     | 0         | 2         | Starting mark, value: 0x5566                                                                                                           |
| **CTRL**    | 2         | 1         | Control byte with the following bits:                                                                                                  |
|             |           |           | - **bit 0**: `need_ack` (if the current package needs acknowledgment)                                                                  |
|             |           |           | - **bit 1**: `ack_pack` (if the package is an acknowledgment package)                                                                  |
|             |           |           | - **bits 2-7**: Reserved                                                                                                               |
| **Data_len**| 3         | 2         | Length of the data field in bytes. The low byte is first.                                                                             |
| **SEQ**     | 5         | 2         | Frame sequence number ranging from 0 to 65535. The low byte is first.                                                                 |
| **CMD_ID**  | 7         | 1         | Command identifier                                                                                                                     |
| **DATA**    | 8         | `Data_len`| The data payload. The length is determined by the `Data_len` field.                                                                    |
| **CRC16**   |   | 2    | CRC16 checksum for the entire data package.                                                                                           |


In [12]:
ser = serial.Serial(port= '/dev/cu.usbmodemN32G45x1',
                    baudrate= 57600,
                    timeout= 1
                    )
data2=  '55 66 01 01 00 00 00 42 02 B5 C0' # 2 HZ
# data0 = '55 66 01 01 00 00 00 42 00 F7 E0' # 0 HZ

if not ser.is_open:
    ser.open()

ser.write(bytes.fromhex(data2))

# res1 = ser.read(ser.in_waiting).hex()
res1 = ser.readline().hex()
# res1 = res1.hex()
print('Got the following response:\n', res1)

# ser.write(bytes.fromhex(data2))  
# res2 = ser.read(ser.in_waiting)
# print(ser.in_waiting)
# print(res2.hex())

# ser.close()


Got the following response:
 dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04c8485566002000862842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a0497e15566002000872842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04a2865566002000882842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04c3aa5566002000892842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04f6cd55660020008a2842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04a96455660020008b2842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a049c0355660020008c2842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04362655660020008d2842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04034155660020008e2842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a045ce855660020008f2842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8031a041a04698f5566002000902842dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e80

In [14]:

res1 = ser.read(2).hex()
# res1 = res1.hex()
print('Got the following response:\n', res1)

Got the following response:
 5566


In [177]:
pattern = re.compile(r'5566.{0,80}', re.IGNORECASE)

# match = pattern.search(res1)

# match.group()
pattern.findall(res1)

['55660020002f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044f14',
 '5566002000305142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b82b',
 '5566002000315142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048d4c',
 '5566002000325142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d2e5',
 '5566002000335142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e782',
 '5566002000345142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044da7',
 '5566002000355142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0478c0',
 '5566002000365142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042769',
 '5566002000375142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04120e',
 '5566002000385142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047322',
 '5566002000395142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044645',
 '55660020003a5142dc0

In [191]:
# Find '5566' followed by any two characters and capture the next four characters
pattern = re.compile(r'5566..(.{4})')

# Find all matches
matches = pattern.findall(res1)
matches

['2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 '2000',
 

In [205]:
data_len = 64

pattern_string = fr'5566.{{12}}.{{{data_len}}}.{{4}}'
pattern = re.compile(pattern_string)
matches = pattern.findall(res1)
matches

['55660020002f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044f14',
 '5566002000305142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b82b',
 '5566002000315142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048d4c',
 '5566002000325142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d2e5',
 '5566002000335142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e782',
 '5566002000345142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044da7',
 '5566002000355142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0478c0',
 '5566002000365142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042769',
 '5566002000375142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04120e',
 '5566002000385142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047322',
 '5566002000395142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044645',
 '55660020003a5142dc0

In [189]:
byte_value = bytes.fromhex('0020')
decimal_value = int.from_bytes(byte_value)
print('Data length in bytes:',decimal_value)  
print('Data length in hex:',decimal_value *2)

Data length in bytes: 32
Data length in hex: 64


In [194]:
q = '2000'

q = q[2]+q[3]+q[0]+q[1]
q

'0020'

In [12]:
def main():

    ser = serial.Serial(port= '/dev/cu.usbmodemN32G45x1',
                    baudrate= 57600,
                    timeout= 1
                    )
    data2=  '55 66 01 01 00 00 00 42 02 B5 C0' # 2 HZ

    if not ser.is_open:
        ser.open()

    ser.write(bytes.fromhex(data2))

    # res1 = ser.read(ser.in_waiting).hex()
    res1 = ser.readline().hex()
    # res1 = res1.hex()
    print('Got the following response:\n', res1)

    data = data_extractor(res1)

    return data

In [7]:
def datalen_finder(ser : str) -> List[str]:
    ''' 
    This function extracts the initial-raw length from the stream of input data.
    '''

    # Find '5566' followed by any two characters and capture the next four characters
    pattern = re.compile(r'5566..(.{4})')

    # Find all matches
    matches = pattern.findall(ser)
    
    return matches

In [8]:
def datalen_modifier(len_hex : str) -> str:

    '''
    Data length in hex format, low byte (LSB) in front based on datasheet. 
    This function modifies this to MSB in front
    '''

    len = len_hex[2]+len_hex[3]+len_hex[0]+len_hex[1]

    return len

In [4]:
def datalen_determiner(revised_len : str) -> int:

    ''' 
    This function finds the data length in hex. 
    '''
    byte_value = bytes.fromhex('0020')
    decimal_value = int.from_bytes(byte_value)
    print('Data length in bytes:',decimal_value)  
    print('Data length in hex:',decimal_value *2)

    return decimal_value *2


In [10]:
def data_extractor(ser : str) -> str: 

    ''' 
    Extracts data from a sequence of serial hexadecimal characters
    '''

    for raw_len in datalen_finder (ser):

        revered = datalen_modifier (raw_len)
        data_len = datalen_determiner(revered)
        pattern_string = fr'5566.{{12}}.{{{data_len}}}.{{4}}'
        pattern = re.compile(pattern_string)
        matches = pattern.findall(res1)
        
        return matches

In [ ]:
# Defining dataframe structure

start_frame = '5566'
crtl_len = 2
expected_data_len = 4
frame_seq_len = 4
mcd_id_len = 2
crc16_len = 4

In [156]:
import re
import serial

ser = serial.Serial(port= '/dev/cu.usbmodemN32G45x1',
                    baudrate= 57600,
                    timeout= 1
                    )

data2=  '55 66 01 01 00 00 00 42 02 B5 C0' # 2 HZ

pattern = re.compile(r'5566.*?b5c0', re.IGNORECASE)


def find_pattern_in_stream():
    buffer = ""
    
    ser.write(bytes.fromhex(data2))

    if not ser.is_open:
        ser.open()

    while True:
        
        data = ser.read(1000)

        if not data:
            continue
        
        hex_data = data.hex()
        
        buffer += hex_data
        
        match = pattern.search(buffer)

        if match:

            matched_data = match.group()
            print(f"Here is your pattern: {matched_data}")
            

try:
    find_pattern_in_stream()
except KeyboardInterrupt:
    print("Stopped by user.")
finally:
    ser.close()

Stopped by user.


In [178]:
len('dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04')

64

Should be = \
55 66 00 20 00 99 00 42 DC 05 DC 00 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 1A 04 DC 05 DC 05 1A 04 1A 04 FF 88

In [168]:
text = '55 66 00 20 00 99 00 42 DC 05 DC 00 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 DC 05 1A 04 DC 05 DC 05 1A 04 1A 04 FF 88'
text = text.replace(" ", "").lower()
datalen = len(text)
datalen

84

In [52]:

# Which port? what is the baud rate?
serial_port = '/dev/cu.usbmodemN32G45x1'
baud_rate = 9600

# Opening the serial port
ser = serial.Serial(port=serial_port, 
                    baudrate=baud_rate,
                    # stopbits=serial.STOPBITS_ONE,
                    timeout=1)

if ser:

    try:
        while True:
            if ser.in_waiting > 0:  # Is there some data waiting??
                data = ser.readline().decode('utf-8').strip() # Decoding bytes to string, strip whitespace
                print(data)
            time.sleep(0.1)  # Add a small delay to prevent high CPU usage

    # Keyboard interupt included
    except serial.SerialTimeoutException:
        print("Read operation timed out")

    except serial.SerialException as e:
        print(f"Serial error: {e}")

    except KeyboardInterrupt:
        print("Keyboard interrupt recieved")

    finally:
        if ser and ser.is_open:   
            ser.close()



Keyboard interrupt recieved


In [73]:
CRC16_XMODEM_TABLE = [
        0x0000, 0x1021, 0x2042, 0x3063, 0x4084, 0x50a5, 0x60c6, 0x70e7,
        0x8108, 0x9129, 0xa14a, 0xb16b, 0xc18c, 0xd1ad, 0xe1ce, 0xf1ef,
        0x1231, 0x0210, 0x3273, 0x2252, 0x52b5, 0x4294, 0x72f7, 0x62d6,
        0x9339, 0x8318, 0xb37b, 0xa35a, 0xd3bd, 0xc39c, 0xf3ff, 0xe3de,
        0x2462, 0x3443, 0x0420, 0x1401, 0x64e6, 0x74c7, 0x44a4, 0x5485,
        0xa56a, 0xb54b, 0x8528, 0x9509, 0xe5ee, 0xf5cf, 0xc5ac, 0xd58d,
        0x3653, 0x2672, 0x1611, 0x0630, 0x76d7, 0x66f6, 0x5695, 0x46b4,
        0xb75b, 0xa77a, 0x9719, 0x8738, 0xf7df, 0xe7fe, 0xd79d, 0xc7bc,
        0x48c4, 0x58e5, 0x6886, 0x78a7, 0x0840, 0x1861, 0x2802, 0x3823,
        0xc9cc, 0xd9ed, 0xe98e, 0xf9af, 0x8948, 0x9969, 0xa90a, 0xb92b,
        0x5af5, 0x4ad4, 0x7ab7, 0x6a96, 0x1a71, 0x0a50, 0x3a33, 0x2a12,
        0xdbfd, 0xcbdc, 0xfbbf, 0xeb9e, 0x9b79, 0x8b58, 0xbb3b, 0xab1a,
        0x6ca6, 0x7c87, 0x4ce4, 0x5cc5, 0x2c22, 0x3c03, 0x0c60, 0x1c41,
        0xedae, 0xfd8f, 0xcdec, 0xddcd, 0xad2a, 0xbd0b, 0x8d68, 0x9d49,
        0x7e97, 0x6eb6, 0x5ed5, 0x4ef4, 0x3e13, 0x2e32, 0x1e51, 0x0e70,
        0xff9f, 0xefbe, 0xdfdd, 0xcffc, 0xbf1b, 0xaf3a, 0x9f59, 0x8f78,
        0x9188, 0x81a9, 0xb1ca, 0xa1eb, 0xd10c, 0xc12d, 0xf14e, 0xe16f,
        0x1080, 0x00a1, 0x30c2, 0x20e3, 0x5004, 0x4025, 0x7046, 0x6067,
        0x83b9, 0x9398, 0xa3fb, 0xb3da, 0xc33d, 0xd31c, 0xe37f, 0xf35e,
        0x02b1, 0x1290, 0x22f3, 0x32d2, 0x4235, 0x5214, 0x6277, 0x7256,
        0xb5ea, 0xa5cb, 0x95a8, 0x8589, 0xf56e, 0xe54f, 0xd52c, 0xc50d,
        0x34e2, 0x24c3, 0x14a0, 0x0481, 0x7466, 0x6447, 0x5424, 0x4405,
        0xa7db, 0xb7fa, 0x8799, 0x97b8, 0xe75f, 0xf77e, 0xc71d, 0xd73c,
        0x26d3, 0x36f2, 0x0691, 0x16b0, 0x6657, 0x7676, 0x4615, 0x5634,
        0xd94c, 0xc96d, 0xf90e, 0xe92f, 0x99c8, 0x89e9, 0xb98a, 0xa9ab,
        0x5844, 0x4865, 0x7806, 0x6827, 0x18c0, 0x08e1, 0x3882, 0x28a3,
        0xcb7d, 0xdb5c, 0xeb3f, 0xfb1e, 0x8bf9, 0x9bd8, 0xabbb, 0xbb9a,
        0x4a75, 0x5a54, 0x6a37, 0x7a16, 0x0af1, 0x1ad0, 0x2ab3, 0x3a92,
        0xfd2e, 0xed0f, 0xdd6c, 0xcd4d, 0xbdaa, 0xad8b, 0x9de8, 0x8dc9,
        0x7c26, 0x6c07, 0x5c64, 0x4c45, 0x3ca2, 0x2c83, 0x1ce0, 0x0cc1,
        0xef1f, 0xff3e, 0xcf5d, 0xdf7c, 0xaf9b, 0xbfba, 0x8fd9, 0x9ff8,
        0x6e17, 0x7e36, 0x4e55, 0x5e74, 0x2e93, 0x3eb2, 0x0ed1, 0x1ef0,
        ]

def crc16_cal(data, length):
    crc = 0x0000
    for i in range(length):
        temp = (crc >> 8) & 0xFF
        crc = ((crc << 8) & 0xFFFF) ^ CRC16_XMODEM_TABLE[(data[i] ^ temp) & 0xFF]
        # print(crc)

    
    crc = hex(crc)[2:]
    crc_byte1 = crc[:2]
    crc_byte2 = crc[2:]
    crc_reversed = crc_byte2 + crc_byte1
    # print(crc)
    return crc_reversed


In [74]:

import re

data = 'dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047a7355660020002f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044f145566002000305142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b82b5566002000315142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048d4c5566002000325142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d2e55566002000335142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e7825566002000345142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044da75566002000355142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0478c05566002000365142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0427695566002000375142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04120e5566002000385142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0473225566002000395142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04464555660020003a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0419ec55660020003b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042c8b55660020003c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0486ae55660020003d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b3c955660020003e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04ec6055660020003f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d9075566002000405142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a045a515566002000415142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a046f365566002000425142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04309f5566002000435142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0405f85566002000445142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04afdd5566002000455142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a049aba5566002000465142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04c5135566002000475142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04f0745566002000485142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0491585566002000495142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04a43f55660020004a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04fb9655660020004b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04cef155660020004c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0464d455660020004d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0451b355660020004e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a040e1a55660020004f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a043b7d5566002000505142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04cc425566002000515142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04f9255566002000525142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04a68c5566002000535142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0493eb5566002000545142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0439ce5566002000555142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a040ca95566002000565142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0453005566002000575142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0466675566002000585142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04074b5566002000595142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04322c55660020005a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a046d8555660020005b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0458e255660020005c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04f2c755660020005d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04c7a055660020005e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04980955660020005f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04ad6e5566002000605142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0476765566002000615142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0443115566002000625142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a041cb85566002000635142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0429df5566002000645142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0483fa5566002000655142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b69d5566002000665142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e9345566002000675142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04dc535566002000685142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04bd7f5566002000695142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04881855660020006a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d7b155660020006b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e2d655660020006c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0448f355660020006d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047d9455660020006e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04223d55660020006f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04175a5566002000705142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e0655566002000715142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d5025566002000725142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048aab5566002000735142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04bfcc5566002000745142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0415e95566002000755142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04208e5566002000765142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047f275566002000775142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044a405566002000785142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042b6c5566002000795142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a041e0b55660020007a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0441a255660020007b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0474c555660020007c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04dee055660020007d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04eb8755660020007e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b42e55660020007f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0481495566002000805142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b2835566002000815142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0487e45566002000825142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d84d5566002000835142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04ed2a5566002000845142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04470f5566002000855142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0472685566002000865142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042dc15566002000875142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0418a65566002000885142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04798a5566002000895142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044ced55660020008a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04134455660020008b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04262355660020008c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048c0655660020008d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b96155660020008e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e6c855660020008f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d3af5566002000905142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0424905566002000915142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0411f75566002000925142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044e5e5566002000935142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047b395566002000945142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d11c5566002000955142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e47b5566002000965142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04bbd25566002000975142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048eb55566002000985142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04ef995566002000995142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04dafe55660020009a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04855755660020009b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b03055660020009c5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a041a1555660020009d5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042f7255660020009e5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0470db55660020009f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0445bc5566002000a05142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a049ea45566002000a15142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04abc35566002000a25142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04f46a5566002000a35142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04c10d5566002000a45142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a046b285566002000a55142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a045e4f5566002000a65142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0401e65566002000a75142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0434815566002000a85142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0455ad5566002000a95142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0460ca5566002000aa5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a043f635566002000ab5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a040a'


buffer = ''
def datalen_determiner(revised_len : str) -> int:

    ''' 
    This function finds the data length in hex. 
    '''
    byte_value = bytes.fromhex(f'{revised_len}')
    decimal_value = int.from_bytes(byte_value)
    # print('Data length in bytes:',decimal_value)  
    # print('Data length in hex:',decimal_value *2)

    return decimal_value *2

try: 
    while True:
        # Finding the length of the data
        pattern_string = r'5566..(.{4})' 
        pattern = re.compile(pattern_string, re.IGNORECASE)
        match = pattern.search(data)  # returns None if not match found


        if match:

            # Adjusting the data length
            len_hex = match.group()[-4:]
            len_hex = len_hex[2]+len_hex[3]+len_hex[0]+len_hex[1]
            data_len = datalen_determiner(len_hex)

            # Reading the first match based on data length
            pattern_string2 = fr'5566.{{12}}.{{{data_len}}}.{{4}}'    
            pattern2 = re.compile(pattern_string2, re.IGNORECASE)
            matches = pattern2.search(data)

            if matches:
                
                # Verifying the CRC16
                found_data = matches.group()
                recieved_crc16 = found_data[-4:]

                data_ = found_data[:len(found_data)-4]  # Data without checksum
                byte_list = [int(data_[i:i+2], 16) for i in range(0, len(data_), 2)] # Convert to a list of integer values
                crc_calculated = crc16_cal(byte_list, len(byte_list))

                if recieved_crc16 == crc_calculated:
                
                    buffer += found_data
                    # print(buffer)

                    # print(found_data)
                    data = data.replace(found_data,'')
                else:
                    data = data.replace(found_data,'')

            else:
                break
        
        if buffer:
            print(buffer)
            buffer = ''

except KeyboardInterrupt:
    print("Keyboard interrupt recieved")
    
# print(buffer)


55660020002f5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044f14
5566002000305142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04b82b
5566002000315142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a048d4c
5566002000325142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04d2e5
5566002000335142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a04e782
5566002000345142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044da7
5566002000355142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0478c0
5566002000365142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a042769
5566002000385142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a047322
5566002000395142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a044645
55660020003a5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05dc05e8031a041a0419ec
55660020003b5142dc05dc05dc05dc05dc051a041a04dc051a041a041a04dc05d

In [42]:
CRC16_XMODEM_TABLE = [
        0x0000, 0x1021, 0x2042, 0x3063, 0x4084, 0x50a5, 0x60c6, 0x70e7,
        0x8108, 0x9129, 0xa14a, 0xb16b, 0xc18c, 0xd1ad, 0xe1ce, 0xf1ef,
        0x1231, 0x0210, 0x3273, 0x2252, 0x52b5, 0x4294, 0x72f7, 0x62d6,
        0x9339, 0x8318, 0xb37b, 0xa35a, 0xd3bd, 0xc39c, 0xf3ff, 0xe3de,
        0x2462, 0x3443, 0x0420, 0x1401, 0x64e6, 0x74c7, 0x44a4, 0x5485,
        0xa56a, 0xb54b, 0x8528, 0x9509, 0xe5ee, 0xf5cf, 0xc5ac, 0xd58d,
        0x3653, 0x2672, 0x1611, 0x0630, 0x76d7, 0x66f6, 0x5695, 0x46b4,
        0xb75b, 0xa77a, 0x9719, 0x8738, 0xf7df, 0xe7fe, 0xd79d, 0xc7bc,
        0x48c4, 0x58e5, 0x6886, 0x78a7, 0x0840, 0x1861, 0x2802, 0x3823,
        0xc9cc, 0xd9ed, 0xe98e, 0xf9af, 0x8948, 0x9969, 0xa90a, 0xb92b,
        0x5af5, 0x4ad4, 0x7ab7, 0x6a96, 0x1a71, 0x0a50, 0x3a33, 0x2a12,
        0xdbfd, 0xcbdc, 0xfbbf, 0xeb9e, 0x9b79, 0x8b58, 0xbb3b, 0xab1a,
        0x6ca6, 0x7c87, 0x4ce4, 0x5cc5, 0x2c22, 0x3c03, 0x0c60, 0x1c41,
        0xedae, 0xfd8f, 0xcdec, 0xddcd, 0xad2a, 0xbd0b, 0x8d68, 0x9d49,
        0x7e97, 0x6eb6, 0x5ed5, 0x4ef4, 0x3e13, 0x2e32, 0x1e51, 0x0e70,
        0xff9f, 0xefbe, 0xdfdd, 0xcffc, 0xbf1b, 0xaf3a, 0x9f59, 0x8f78,
        0x9188, 0x81a9, 0xb1ca, 0xa1eb, 0xd10c, 0xc12d, 0xf14e, 0xe16f,
        0x1080, 0x00a1, 0x30c2, 0x20e3, 0x5004, 0x4025, 0x7046, 0x6067,
        0x83b9, 0x9398, 0xa3fb, 0xb3da, 0xc33d, 0xd31c, 0xe37f, 0xf35e,
        0x02b1, 0x1290, 0x22f3, 0x32d2, 0x4235, 0x5214, 0x6277, 0x7256,
        0xb5ea, 0xa5cb, 0x95a8, 0x8589, 0xf56e, 0xe54f, 0xd52c, 0xc50d,
        0x34e2, 0x24c3, 0x14a0, 0x0481, 0x7466, 0x6447, 0x5424, 0x4405,
        0xa7db, 0xb7fa, 0x8799, 0x97b8, 0xe75f, 0xf77e, 0xc71d, 0xd73c,
        0x26d3, 0x36f2, 0x0691, 0x16b0, 0x6657, 0x7676, 0x4615, 0x5634,
        0xd94c, 0xc96d, 0xf90e, 0xe92f, 0x99c8, 0x89e9, 0xb98a, 0xa9ab,
        0x5844, 0x4865, 0x7806, 0x6827, 0x18c0, 0x08e1, 0x3882, 0x28a3,
        0xcb7d, 0xdb5c, 0xeb3f, 0xfb1e, 0x8bf9, 0x9bd8, 0xabbb, 0xbb9a,
        0x4a75, 0x5a54, 0x6a37, 0x7a16, 0x0af1, 0x1ad0, 0x2ab3, 0x3a92,
        0xfd2e, 0xed0f, 0xdd6c, 0xcd4d, 0xbdaa, 0xad8b, 0x9de8, 0x8dc9,
        0x7c26, 0x6c07, 0x5c64, 0x4c45, 0x3ca2, 0x2c83, 0x1ce0, 0x0cc1,
        0xef1f, 0xff3e, 0xcf5d, 0xdf7c, 0xaf9b, 0xbfba, 0x8fd9, 0x9ff8,
        0x6e17, 0x7e36, 0x4e55, 0x5e74, 0x2e93, 0x3eb2, 0x0ed1, 0x1ef0,
        ]

def crc16_cal(data : List[int], length : int) -> str:
    ''' 
    data : List of bytes
    length : Length of data 
    '''
    crc = 0x0000
    for i in range(length):
        temp = (crc >> 8) & 0xFF
        crc = ((crc << 8) & 0xFFFF) ^ CRC16_XMODEM_TABLE[(data[i] ^ temp) & 0xFF]
    
    crc = hex(crc)[2:]
    crc_byte1 = crc[:2]  # crc16 might not always be 4 characters! Need to reverse it byte by byte
    crc_byte2 = crc[2:]
    crc_reversed = crc_byte2 + crc_byte1
    return crc_reversed


In [52]:
ser = serial.Serial(port= '/dev/cu.usbmodemN32G45x1',
                    baudrate= 57600,
                    timeout= 1
                    )
data2=  '55 66 01 01 00 00 00 42 02 B5 C0' # 2 HZ

if not ser.is_open:
    ser.open()

ser.write(bytes.fromhex(data2))

# ----------------------------------------------------------------
def datalen_finder(data_stream : str) -> str:
    ''' 
    This function extracts the initial-raw length from the stream of input data.
    '''
    # Find '5566' followed by any two characters and capture the next four characters
    pattern_string = r'5566..(.{4})'
    pattern = re.compile(pattern_string, re.IGNORECASE)
    # Find the first matche
    match = pattern.search(data_stream)  
    return match.group()[-4:]

def reverse(input : str) -> str:
    '''
    This function resolves the LSM front issue in the datasheet, putting MSB in front
    '''
    input_byte1 = input[:2]  
    input_byte2 = input[2:]
    reversed = input_byte2 + input_byte1
    return reversed


def datalen_hex(revised_len : str) -> int:
    ''' 
    This function calculates the data length in hexadecimal characters. 
    '''
    byte_value = bytes.fromhex(f'{revised_len}')
    decimal_value = int.from_bytes(byte_value)
    # print('Data length in bytes:',decimal_value)  
    # print('Data length in hex:',decimal_value *2)
    return decimal_value *2

# ----------------------------------------------------------------

buffer = ''

while True:

    buffer += ser.read(1).hex()
    # print(buffer)

    if buffer == '55':
        buffer += ser.read(1).hex()
    else:
         buffer = ''

    if buffer == '5566':
            buffer += ser.read(6).hex()
            data_len = datalen_hex(reverse(datalen_finder(buffer)))
            data_len /= 2
            # print(type(data_len)==float)
            # print(data_len)
            buffer += ser.read(int(data_len)).hex()
            # print(buffer)
            buffer += ser.read(2).hex()
    else:
         buffer = ''

    data = buffer[:len(buffer)-4]
    recieved_crc16 = buffer[-4:]
    # Converting data to a list of integer values to calculate checksum for it
    byte_list = [int(data[i:i+2], 16) for i in range(0, len(data), 2)] 
    crc_calculated = crc16_cal(byte_list, len(byte_list))
    # Verifying data validity, updating data_stream and buffer registers
    if recieved_crc16 == crc_calculated and len(buffer)> int(data_len) :
        print(buffer)
        buffer = ''
    else:
         print('Received data has invalid CRC!')
         buffer = ''

    # print('Got the following response:\n', data)

Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
Received data has invalid CRC!
5566002000e95642dc05dc05dc05dc059e071a041a04dc051a041a041a04dc05dc05e8

KeyboardInterrupt: 

In [38]:
len('5566002000362842dc05dc05660674069e071a041a04dc051a041a041a04dc05dc05e8031a041a049df1')

84

In [51]:
data = 'dc05dc05660674069e071a041a04dc051a041a041a04dc05dc05e8031a041a04'
len(data)

64

In [67]:
import numpy as np
channel_data_dict = {}
channel_data_list = []
num_of_channels = 16
i = 0
n = 0
while i <= num_of_channels-1:

    # channel_data[f'CH[{i}]'] = data[n:n+4]
    channel_data_dict[f'CH[{i}]'] = int(data[n:n+4],16) 
    n+=4
    i += 1
else:
    i = 0
    n = 0
    
channel_data_list = list(channel_data_dict.values())

channel_data_dict

{'CH[0]': 56325,
 'CH[1]': 56325,
 'CH[2]': 26118,
 'CH[3]': 29702,
 'CH[4]': 40455,
 'CH[5]': 6660,
 'CH[6]': 6660,
 'CH[7]': 56325,
 'CH[8]': 6660,
 'CH[9]': 6660,
 'CH[10]': 6660,
 'CH[11]': 56325,
 'CH[12]': 56325,
 'CH[13]': 59395,
 'CH[14]': 6660,
 'CH[15]': 6660}

In [65]:
desired_idx = [13,6,0,4,3,8,9,12,10,15,14,1,7,11,1,2]

out_list = np.zeros(16)

for in_idx,out_idx in enumerate(desired_idx):
    
    out_list[in_idx] =  channel_data_list[out_idx] 

out_list[15]

np.float64(26118.0)

In [66]:
def ch_data_transformer (ch_data_dict : dict, desired_idx : list[int]) -> list[int] :

    # channel_data_list = []
    channel_data_list = list(ch_data_dict.values())

    out_list = np.zeros(16)

    for in_idx,out_idx in enumerate(desired_idx):
        
        out_list[in_idx] =  channel_data_list[out_idx] 

    return out_list


In [68]:
ch_data_transformer(channel_data_dict,desired_idx)

array([59395.,  6660., 56325., 40455., 29702.,  6660.,  6660., 56325.,
        6660.,  6660.,  6660., 56325., 56325., 56325., 56325., 26118.])

In [18]:
from collections import defaultdict
def ch_data_transformer(in_ch_data : dict) -> dict:

    out_ch_data = defaultdict(int)

    for key in in_ch_data:
        out_ch_data[key] 

    # for key,value in in_ch_data.items():

    return out_ch_data

ch_data_transformer(channel_data)



defaultdict(int,
            {'CH[1]': 0,
             'CH[2]': 0,
             'CH[3]': 0,
             'CH[4]': 0,
             'CH[5]': 0,
             'CH[6]': 0,
             'CH[7]': 0,
             'CH[8]': 0,
             'CH[9]': 0,
             'CH[10]': 0,
             'CH[11]': 0,
             'CH[12]': 0,
             'CH[13]': 0,
             'CH[14]': 0,
             'CH[15]': 0,
             'CH[16]': 0})

In [ ]:
#!/usr/bin/env python3
import rospy
from geometry_msgs.msg import Twist

def publish_joystick_data():
    rospy.init_node('joystick_simulator', anonymous=True)
    pub = rospy.Publisher('/drone/cmd_vel', Twist, queue_size=10)
    rate = rospy.Rate(10)  # Hz

    while not rospy.is_shutdown():
        twist = Twist()
        # Simulate joystick axes (e.g., throttle, pitch, roll, yaw)
        twist.linear.x = 0.5  # Forward/backward
        twist.linear.y = 0.0  # Left/right
        twist.linear.z = 0.1  # Up/down
        twist.angular.z = 0.3  # Yaw left/right

        pub.publish(twist)
        rate.sleep()

if _name_ == '_main_':
    try:
        publish_joystick_data()
    except rospy.ROSInterruptException:
        pass